## Additional End of week Exercise - week 2

This includes 
- Gradio UI
- use of the system prompt to add expertise
- audio input so you can talk to it
- respond with audio

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import gradio as gr
import json

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [3]:
system_message = """You are a personal study tutor, designed to provide clear, yet brief and succint answers to 
students that ask you questions. The topics are related to data science, computer science 
and technology in general, so you are allowed to use a moderate level of jargon. Explain in 
simple terminology, so a student can easily understand. 

You may also be asked about prices for special courses.In this case, respond that you have no such
data available. 

"""
# Use a tabular format where possible 
# for ease of information flow 

In [4]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history

In [5]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 7.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.2/26.2 MB 3.9 MB/s eta 0:00:0000:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803405 sha256=d2d25219c1fc903e1f27307bd441251187c8d7c2c8cc6833300bc46b967c1dc4
  Stored in directory: /Users/triimamwicaksono/Library/Caches/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper


In [6]:
import whisper

def transcribe_audio(audio):
    if audio is None:
        return "No audio received."
    
    model = whisper.load_model("base")  # You can use "tiny", "small", etc.
    result = model.transcribe(audio)
    
    return result["text"]

In [7]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

In [8]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=400,type="messages")
    with gr.Row():
        entry = gr.Textbox(label="Chat with our StudyAI Assistant:")
    # with gr.Row():
    #     entry = gr.Textbox(label="Speak or Type:", placeholder="Speak your question...", interactive=True, microphone=True)
    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Speak your question")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    def handle_audio(audio, history):
        text = transcribe_audio(audio)
        history += [{"role": "user", "content": text}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=[chatbot], outputs=[chatbot]
    )

    audio_input.change(handle_audio, inputs=[audio_input, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=[chatbot], outputs=[chatbot]
    )
    
    clear.click(lambda: [], inputs=None, outputs=chatbot, queue=False)

In [9]:
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
